# General figures

In [ ]:
from cellassign import assign_cats
import gzip
import itertools as itl
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import os
import pandas as pd
import scanpy as sc
import scanpy.external as sce
import seaborn as sns
import triku as tk
import itertools
import matplotlib.pylab as pylab

In [ ]:
from functions import adata_plot_KOvsWT, dict_WT_KO_colors, adata_plot_KOvsWT_2bars

In [ ]:
from datetime import date

today = date.today()
today = today.strftime("%Y_%m_%d")

In [ ]:
# Palettes for UMAP gene expression

magma = [plt.get_cmap('magma')(i) for i in np.linspace(0,1, 80)]
magma[0] = (0.88, 0.88, 0.88, 1)
magma = mpl.colors.LinearSegmentedColormap.from_list("", magma[:65])

In [ ]:
params = {'figure.dpi': 200,
          'legend.fontsize': 'large',
         'axes.labelsize': 'x-large',
         'axes.titlesize':'x-large',
         'xtick.labelsize':'x-large',
         'ytick.labelsize':'x-large', 
         'axes.grid': False}
pylab.rcParams.update(params)
# mpl.style.use('default')

pd.set_option('display.max_columns', None)

## Functions and variables

In [ ]:
seed = 0
%store seed

## Load adata

In [ ]:
adata_all_harmony = sc.read_h5ad('adatas/adata_all_harmony.h5')
sc.tl.umap(adata_all_harmony, min_dist=0.7, random_state=seed)

## Figure of main populations in 4 panels

In [ ]:
# Join al subpopulations into one
major_merged_types = ['Fibroblast', 'Keratinocyte', 'Neutrophil', 'Vascular cell', 'T cell']

In [ ]:
merged_cell_types = adata_all_harmony.obs['cell_type_whole'].astype(str)

for mct in major_merged_types:
    mask = [mct in i for i in adata_all_harmony.obs['cell_type_whole']]
    merged_cell_types[mask] = mct

merged_cell_types[[('B cell' in i) or ('T cell' in i) or ('NK cell' in i) or 
                   ('Plasma cell' in i) for i in adata_all_harmony.obs['cell_type_whole']]] = 'Lymphocyte' 
merged_cell_types[[('Fibroblast' in i) or ('Skeletal' in i) or 
                   ('Melanocyte' in i) for i in adata_all_harmony.obs['cell_type_whole']]] = 'Fibroblast' 
merged_cell_types[[('Schwann' in i) or ('Glial' in i) for i in adata_all_harmony.obs['cell_type_whole']]] = 'Schwann cell' 
merged_cell_types[[('Monocyte' in i) or ('Macrophage' in i) or 
                   ('Langerhans cell' in i) for i in adata_all_harmony.obs['cell_type_whole']]] = 'APC' 
merged_cell_types[['Perivascular' in i for i in adata_all_harmony.obs['cell_type_whole']]] = 'Perivascular cell' 

adata_all_harmony.obs['merged_cell_type'] = merged_cell_types
adata_all_harmony.obs['merged_cell_type'].astype('category')

dict_rename = {'Keratinocyte': '0: KRT', 
               'Fibroblast': '1: FB', 
               'Chondrogenic fibroblast': '2: CHFB',
               'Vascular endothelial cell': '3: VEC', 
               'Lymphatic endothelial cell': '4: LEC',
               'Perivascular cell': '5: PVC', 
               'Schwann cell': '6: SCH',
               'Mast cell': '7: MC',
               'Lymphocyte': '8: LYM', 
               'Neutrophil': '9: NEU',
               'APC': '10: APC'}

adata_all_harmony.obs['merged_cell_type'] = [dict_rename[i] for i in adata_all_harmony.obs['merged_cell_type']]
adata_all_harmony.obs['merged_cell_type'] = adata_all_harmony.obs['merged_cell_type'].astype('category')
adata_all_harmony.obs['merged_cell_type'].cat.reorder_categories(list(dict_rename.values()), inplace=True)

adata_all_harmony.obs['merged_cell_type_name'] = [i.split(': ') [1] for i in adata_all_harmony.obs['merged_cell_type']]
adata_all_harmony.obs['merged_cell_type_name'] = adata_all_harmony.obs['merged_cell_type_name'].astype('category')
adata_all_harmony.obs['merged_cell_type_name'].cat.reorder_categories([i.split(': ') [1] for i in dict_rename.values()], inplace=True)

adata_all_harmony.obs['merged_cell_type_number'] = [i.split(': ') [0] for i in adata_all_harmony.obs['merged_cell_type']]
adata_all_harmony.obs['merged_cell_type_number'] = adata_all_harmony.obs['merged_cell_type_number'].astype('category')
adata_all_harmony.obs['merged_cell_type_number'].cat.reorder_categories([i.split(': ') [0] for i in dict_rename.values()], inplace=True)


dict_cell_types_colors = {'0: KRT': '#308914',
                          '6: SCH': '#d3bc5f',
                          
                          '1: FB': '#A0302B',
                          '7: MC': '#4d0765',
                          
                          '2: CHFB': "#C97C79",
                          '8: LYM': '#8e0c6f',
                          
                          '3: VEC': '#d40055',
                          '9: NEU': '#E179ED',
                          
                          '4: LEC': '#ff5599',
                          '10: APC': '#47bcd1',
                          
                          '5: PVC': '#ff9955', 
                    }

list_cell_types = list(dict_cell_types_colors.keys())[::2] + list(dict_cell_types_colors.keys())[1::2]

adata_all_harmony.uns['merged_cell_type_colors'] = [dict_cell_types_colors[i] for i in adata_all_harmony.obs['merged_cell_type'].cat.categories]
adata_all_harmony.uns['merged_cell_type_number_colors'] = adata_all_harmony.uns['merged_cell_type_colors']

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(12, 8))
s = 10
plt.tight_layout()
sc.pl.umap(adata_all_harmony[adata_all_harmony.obs['batch'] == 'KO1'], color='merged_cell_type', s=s, ax = axs[0, 0], legend_loc=False, title='KO1', frameon=False, show=False)
sc.pl.umap(adata_all_harmony[adata_all_harmony.obs['batch'] == 'KO2'], color='merged_cell_type', s=s, ax = axs[0, 1], legend_loc=False, title='KO2', frameon=False, show=False)

sc.pl.umap(adata_all_harmony[adata_all_harmony.obs['batch'] == 'WT1'], color='merged_cell_type', s=s, ax = axs[1, 0], legend_loc=False, title='WT1', frameon=False, show=False)
sc.pl.umap(adata_all_harmony[adata_all_harmony.obs['batch'] == 'WT2'], color='merged_cell_type', s=s, ax = axs[1, 1], legend_loc=False, title='WT2', frameon=False,  show=False)

plt.subplots_adjust(wspace=0.025, hspace=0.1)
plt.savefig(f'results/figures/makers_4_panels_{today}.png', dpi=300)


fig2, ax = plt.subplots(1, 1, figsize=(25, 3))
plt.axis('off')
legend_elements = [mpl.lines.Line2D([0], [0], marker='o', color=(1, 1, 1, 1), label=name_i, markerfacecolor=color_i, markersize=25) for name_i, color_i in dict_cell_types_colors.items()]
ax.legend(handles=legend_elements, loc='upper left', ncol=6, frameon=False, fontsize=25)
plt.tight_layout()
# plt.savefig(f'results/figures/makers_4_panels_legend_{today}.jpg', dpi=300)
plt.show()

## Figure of main populations in 2 panels (WT / KO)

In [ ]:
adata_all_harmony.obs['conditon-cell_type'] = (adata_all_harmony.obs['merged_cell_type_name'].astype(str) + '-' + adata_all_harmony.obs['condition'].astype(str)).astype('category')
adata_all_harmony.uns['conditon-cell_type_colors'] = list(itertools.chain(*list(zip(adata_all_harmony.uns['merged_cell_type_colors'], adata_all_harmony.uns['merged_cell_type_colors']))))

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(12, 4))
s = 14
plt.tight_layout()
sc.pl.umap(adata_all_harmony[adata_all_harmony.obs['condition'] == 'KO'], color='merged_cell_type_number', s=s, ax = axs[0], 
           legend_loc='on data', frameon=False, show=False, 
           legend_fontoutline=4, legend_fontsize=14)

title_obj = axs[0].set_title('KO',fontweight='bold', fontsize='xx-large') 
plt.setp(title_obj, color=dict_WT_KO_colors['KO'])

sc.pl.umap(adata_all_harmony[adata_all_harmony.obs['condition'] == 'WT'], color='merged_cell_type_number', s=s, ax = axs[1], 
           legend_loc='on data', frameon=False, show=False, 
           legend_fontoutline=4, legend_fontsize=14)
sc.pl.umap(adata_all_harmony[adata_all_harmony.obs['condition'] == 'WT'], color='merged_cell_type', s=s, ax = axs[1], 
           frameon=False, show=False)

title_obj = axs[1].set_title('WT',fontweight='bold', fontsize='xx-large') 
plt.setp(title_obj, color=dict_WT_KO_colors['WT'])

plt.subplots_adjust(wspace=0.025, hspace=0.1)
plt.tight_layout()

plt.savefig(f'results/figures/Fig2A_{today}.jpg', dpi=300)
plt.savefig(f'results/figures/Fig2A.jpg', dpi=300)

## UMAP with cell types and batches

In [ ]:
adata_all_harmony.uns['batch_colors'] = list(dict_WT_KO_colors.values())
fig, axs = plt.subplots(1, 2, figsize=(12, 4))
s = 10
plt.tight_layout()
sc.pl.umap(adata_all_harmony, color='merged_cell_type', s=s, ax = axs[0], legend_loc=False, title='cell type', frameon=False, show=False)
sc.pl.umap(adata_all_harmony, color='batch', s=s, ax = axs[1], title='batch', frameon=False, show=False)


# plt.subplots_adjust(wspace=0.025, hspace=0.1)
# plt.savefig(f'results/figures/UMAP_all_cell_type_batch{today}.png', dpi=300)

## Proportion of cell types within WT and KO

In [ ]:
adata_plot_KOvsWT(adata_all_harmony, list(dict_cell_types_colors.keys()))

In [ ]:
adata_plot_KOvsWT_2bars(adata_all_harmony, list(dict_cell_types_colors.keys()))
plt.savefig(f'results/figures/Fig2C_{today}.jpg', dpi=300)
plt.savefig(f'results/figures/Fig2C.jpg', dpi=300)

In [ ]:
adata_plot_KOvsWT_2bars(adata_all_harmony, list(dict_cell_types_colors.keys()))
plt.savefig(f'results/figures/Fig2C_{today}.jpg', dpi=300)
plt.savefig(f'results/figures/Fig2C.jpg', dpi=300)

### Main population markers

In [ ]:
cell_markers = {'0: KRT': ['Lgals7', 'S100a14', 'Perp', 'Sfn', 'Dmkn', 'Dsp', 'Ly6d', 'Krt14', 'Dsc3', 'Krt5', 'Pkp3', 'Pkp1', 'Rab25', 'Apoc1', 'Aqp3', 
                                 'Tacstd2', 'Lypd3', 'Krtdap', 'Calm4', 'Krt15', 'Col17a1', 'Serpinb5', 'Fam25c', 'Dsg1a', 'Ckmt1', 'Urah', 'Esrp1', 'Capns2', 
                                 'Trp63', 'Sbsn', 'Dapl1', 'Fgfbp1', 'Krt80', 'Stard10', 'Tfap2a', 'Cldn4', 'Sytl1', 'Celsr2',], 
                '1: FB': ['Bgn', 'Sparc', 'Col6a1', 'Dcn', 'Col1a1', 'Col1a2', 'Mmp2', 'Serpinf1', 'Lum', 'Pcolce', 'Ccdc80', 'Mfap4', 
                         'Mfap2', 'Plac9b', 
                               'Comp', 'Gpc3', 'Ikbip', 'Clec3b',  'Thbs2', 'Igf2', 'Lox', 'Prrx2', 'Ntn1', 'Fkbp7', 'Cyp26b1', 'Apod', 'Lhfp', 'Srpx', 
                               'Cpq', 'Creb3l1', 'Cthrc1', 'P3h3', 'Crtap', 'Gpx7'], 
                '2: CHFB': ['Col2a1', 'Acan', 'Col9a1', 'Col11a1', 'Comp', 'Col9a2', 'Snorc', 'Col11a2', 'Col9a3', 'Fmod', 'Cnmd', 'Mia', 'Ecrg4', 'Chad', 'Ucma', 'Hapln1', 'Scrg1', 'Chadl', 'Msmo1', 'Cspg4', 'Trpv4', 'Plod2', 'Sox5', 'Plagl1'],
                '3: VEC': ['Pecam1', 'Cdh5', 'Arhgap29', 'Adgrf5', 'Emcn', 'Egfl7', 'Rasip1', 'Plvap', 'Eng', 'Cd93', 'Gimap6', 'Fabp4', 'Mast4', 'Gng11', 
                                'Pdlim1', 'Ecscr', 'Ptprb', 'Cldn5', 'Tie1', 'Kitl', 'Depp1', 'Serpine1', 'S1pr1', 'Flt1', 'Neurl3', 'Adgrl4', 'Sox7', 'Aqp1', 
                                'Lrrc32', 'Upp1', 'Pdgfb', 'Robo4', 'Slfn5', 'Icam2', 'Parvb', 'Kdr', 'Podxl', 'Sox17', 'Erg', 'Mmrn2', 'Dll4', ], 
                '4: LEC': ['Fgl2', 'Olfm1', 'Gdf15', 'Mmrn1', 'Prox1', 'Pard6g', 'Ccl21a', 'Lyve1', 'Reln', 'Dtx1', 'Tbx1', 'Sh3gl3', 'Flt4', 'Slc45a3', 
                              'Nts', 'Slc38a4', 'Klhl4', 'Pgm5', 'Flt4', 'Klhl4', 'Slc5a3', 'Pgm5', 'Sgk3', 'Fam189a2', 'Mical2', 'Tbc1d4', 'Clca3a1', 'Scn3a', 'Prkcz',
                              'Cox19', 'Eif2b2', 'Stab1', 'Stmn2', 'Rnf141', 'Tec'], 
                '5: PVC': ['Igfbp7', 'Rgs5', 'Procr', 'Acta2', 'Ndufa4l2', 'Ptp4a3', 'Tagln', 'Aoc3', 'Myh11', 
                                      'Mustn1', 'Apold1', 'Epas1', 'Esam', 'Pdgfrb', 'Myl9', 'Mcam', 'Rgs4', 'Ngf', 'Tinagl1', 
                                      'Ppp1r14a', 'Kcne4', 'Gja4', 'Ednrb', 'Gucy1b1', 'Carmn', 'Gucy1a1', 'Synpo2', 'Abcc9', 'Itga7', 
                                      'Notch3', 'Mylk', 'Actg2', 'Higd1b'],
                '6: SCH': ['Plp1', 'Egfl8', 'Kcna1', 'Mpz', 'Pmp22', 'Cadm4', 'Foxd3', 'Sema3b', 'Mal', 'Gatm', 'Mbp', 'Cnp', 'Limch1', 
                                 'Plekha4', 'Art3', 'Aatk', 'Gas2l3', 'Col28a1', 'Prx', 'Dmd', 'Plekhb1', 'Sox10', 'Utrn', 'Cldn19', 'Ncmap', 
                                 'Mag', 'Zfp536', 'Hspa12a', 'Mlip', 'Itgb8', 'Bcas1', 'Cd59a', 'Cmtm5', 'Pea15a', 'Snca', 'Gjc3', 'Fam178b', 
                                 'Ugt8a', 'Adam23', 'Drp2', 'Kcna2', 'Cadm1'], 
                '7: MC': ['Cma1', 'Tpsb2', 'Mcpt4', 'Hs3st1', 'Tpsab1', 'Tph1', 'Cpa3', 'Gata2', 'Vwa5a', 'Slco2b1', 'Dapp1', 'Cyp11a1', 'Kit', 'Adora3', 'Ap1s3', 'Csf2rb', 'Mrgprb1', 'Sytl3', 'Rab27b', 'Penk', 'Hpgds', 'Ms6st2', 'Mcpt-ps1', 'Lat2', 'Adora2a', 'Calca', 'Slc45a3'],
                '8: LYM': ['Ctla2a', 'Rgs1', 'Cd3d', 'Cd3g', 'Trbc2', 'Ptprcap', 'Neurl3', 'Tnfrsf9', 'Trdv4', 'Trbc1', 'Klrk1', 'Cd3e', 'Cd2', 'Nkg7', 'Il2rg', 
                             'Cd7', 'Tcrg-C1', 'Itk', 'Ctsw', 'Il2rb', 'Ltb', 'Ptpn22', 'Satb1', 'Klrb1b', 'Klrd1', 'Sytl3', 'Lck', 'Sipa1l1', 'Grap2', 'Cd69', 'Camk4'],
                '9: NEU': ['Cstdc4', 'S100a8', 'Csta2', 'S100a9', 'Cstdc5', 'Stfa2l1', 'Wfdc21', 'G0s2', 'Hcar2', 'Csta3', 'Stfa3', 'Lyz2', 'Itgam', 
                               'Retnlg', 'Ngp', 'Pglyrp1', 'Clec4e', 'Camp', 'Hdc', 'C3', 'Cd300lf', 'Hp', 'F10', 'Trem3', 'Mcemp1', 'Chil1', 'Ltf', 'Slc2a3', 
                               'Asprv1', 'Sp140', 'Chil3', 'Msra', 'Cstdc6',],
                '10: APC': ['Tyrobp', 'Ctss', 'Fcer1g', 'Cd74', 'Srgn', 'Basp1', 'H2-Aa', 'Slfn2', 'H2-Ab1', 'Cxcl16', 'H2-Eb1', 'Il7r', 'Csf2rb', 'Bcl2a1b', 'Ncf4', 'Csf1r', 'Malt1', 'Alox5ap', 'Csf2rb2', 'Cd68', 'Pik3r5', 'Pla2g7', 'Ms4a6d', 'Snx20', 'Bcl2a1a', 'Clec2d', 'Cd86' ,'Spi1' 'Lgals3', 'Ctsz'], 
               } 

In [ ]:
sc.tl.rank_genes_groups(adata_all_harmony, groupby='merged_cell_type')

In [ ]:
for cell_type, list_cell_markers in cell_markers.items():
    print(cell_type)
    sc.pl.umap(adata_all_harmony, color=list_cell_markers[:20], alpha=0.5, ncols=5, cmap=magma)

In [ ]:
names = pd.DataFrame(adata_all_harmony.uns['rank_genes_groups']['names'])
pvals = pd.DataFrame(adata_all_harmony.uns['rank_genes_groups']['pvals_adj'])

In [ ]:
# Create a DataFrame with 30 most important genes
list_dfs = []
for cell_type, genes in cell_markers.items():
    df = pd.DataFrame({f'{cell_type} genes': names[cell_type], f'p-value {cell_type}': pvals[cell_type]}, dtype=object)
    df_short = df[df[f'{cell_type} genes'].isin(genes)].sort_values(by=f'p-value {cell_type}', ascending=True).iloc[:50].reset_index(drop=True)
    df[f'p-value {cell_type}'] = df[f'p-value {cell_type}'].apply(lambda x: '%.3E' % x).values.tolist()
    list_dfs.append(df_short)

In [ ]:
df_all_celltypes = pd.concat(list_dfs, axis=1)
df_all_celltypes.to_csv(f'results/{today}-general_population_DEGs.csv', index=False)
df_all_celltypes

## Tracksplot of cell markers

In [ ]:
sc.pl.tracksplot(adata_all_harmony, groupby='merged_cell_type', var_names=df_all_celltypes.iloc[:4,::2].values.transpose().ravel(), 
                 dendrogram=True, xticklabels=False, use_raw=False, ax=ax, show=False)

plt.savefig(f'results/figures/Fig2_Supp1_{today}.jpg', dpi=300)
plt.savefig(f'results/figures/Fig2_Supp1.jpg', dpi=300)

## Dotplot of cell markers

In [ ]:
categories_order = ['KRT', 'FB', 'CHFB', 'VEC', 'LEC', 'PVC', 'SCH', 'MC', 'LYM', 'NEU', 'APC']

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 4))
genes = ['Lgals7', 'S100a14', 'Krtdap', 
         'Dcn', 'Lum', 'Pdgfra', 
         'Col2a1', 'Acan', 'Col11a1', 
         'Cdh5', 'Emcn', 'Cd93',  
         'Prox1', 'Ccl21a', 'Mmrn1', 
         'Rgs5', 'Acta2', 'Myh11',  
         'Plp1', 'Mpz', 'Kcna1', 
         'Cma1', 'Tpsb2', 'Mcpt4', 
         'Ptprcap', 'Cd3d', 'Trbc1', 
         'S100a9', 'G0s2', 'Csta3', 
         'Ctss', 'Cd74', 'H2-Aa']

sc.pl.dotplot(adata_all_harmony, groupby='merged_cell_type_name', var_names=genes, 
              dendrogram=False, show=False, 
             categories_order=categories_order, use_raw=False, ax=ax)

plt.tight_layout()
plt.savefig(f'results/figures/Fig2B_{today}.jpg', dpi=300)
plt.savefig(f'results/figures/Fig2B.jpg', dpi=300)

# Plots of difference of expression

In [ ]:
list_genes_exp = ['Pkm', 'Ldha', 'Ndrg1', 'Neat1', 'Hilpda', 'Hif1a', 'Bnip3', 'Slc2a1']

In [ ]:
# Violin plots
fig, axs = plt.subplots(3, 3, figsize=(16, 12))

order = itl.product(list(adata_all_harmony.obs['merged_cell_type_name'].cat.categories), ['WT', 'KO'])
order = ['-'.join(i) for i in order]

for idx, gene in enumerate(list_genes_exp):
    sc.pl.violin(adata_all_harmony, groupby='conditon-cell_type', keys=gene, rotation=90, order=order,
                 use_raw=False, ax=axs.ravel()[idx], show=False)

    for dirr in ['top', 'right', 'left', 'bottom']:
        axs.ravel()[idx].spines[dirr].set_visible(False)
    axs.ravel()[idx].set_ylabel('')
    axs.ravel()[idx].set_title(gene, fontsize='xx-large')
        
plt.tight_layout()
plt.savefig(f'results/figures/Fig2E_violinplots_{today}.jpg', dpi=300)
# plt.savefig(f'results/figures/Fig2E.png', dpi=300)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 4))

sc.pl.stacked_violin(adata_all_harmony, var_names=list_genes_exp, groupby='conditon-cell_type', 
                 use_raw=False, categories_order=order, swap_axes=True, standard_scale='var', ax=ax, show=False,)

plt.tight_layout()

plt.savefig(f'results/figures/Fig2E_stacked_violinplots_{today}.jpg', dpi=300)
plt.savefig(f'results/figures/Fig2E.jpg', dpi=300)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 4))

sc.pl.stacked_violin(adata_all_harmony, var_names=list_genes_exp, groupby='conditon-cell_type', 
                 use_raw=False, categories_order=order, swap_axes=True, standard_scale='var', row_palette='#454545', ax=ax, show=False,)

plt.tight_layout()

plt.savefig(f'results/figures/Fig2E_stacked_violinplots_gray_{today}.jpg', dpi=300)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 4))

sc.pl.dotplot(adata_all_harmony, var_names=list_genes_exp, groupby='conditon-cell_type', 
                 use_raw=False, categories_order=order, swap_axes=True, show=False, ax=ax)

plt.tight_layout()

plt.savefig(f'results/figures/Fig2E_dotplot_{today}.jpg', dpi=300)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 3))

sc.pl.matrixplot(adata_all_harmony, groupby='conditon-cell_type', var_names=list_genes_exp, dendrogram=False, 
                 swap_axes=True, figsize=(10, 4), categories_order=order, use_raw=False, vmax=2, show=False, ax=ax)

plt.savefig(f'results/figures/Fig2E_matrixplot_{today}.jpg', dpi=300)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 3))

sc.pl.matrixplot(adata_all_harmony, groupby='merged_cell_type_name', var_names=list_genes_exp, dendrogram=False, 
                 swap_axes=True, figsize=(10, 4), categories_order=categories_order, use_raw=False, vmax=2, show=False, ax=ax)

plt.savefig(f'results/figures/Fig2E_matrixplot_cell_type_{today}.jpg', dpi=300)

# Save adata

In [ ]:
adata_all_harmony.write_h5ad('adatas/adata_all_harmony.h5')